# Code Works
This notebook contains examples of LLMs doing the following:
- Add comments and docstrings to Python code
- Write unit test cases for given functions

LLMs used include:
- OpenAI ChatGPT-4o
- Anhropic Clause 3.5 Sonnet
- Google Gemini

The above will be developed as Gradio apps so we could try them out. Let's go!

In [ ]:
# imports
import os
import io
import sys
import textwrap
import datetime

from dotenv import load_dotenv

from openai import OpenAI
import google.generativeai as genai
import anthropic

from IPython.display import Markdown, display, update_display
import gradio as gr

# environment
load_dotenv(override=True)
OPENAI_API_KEY    = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
GOOGLE_API_KEY    = os.getenv('GOOGLE_API_KEY')

# initialize
openai = OpenAI(api_key=OPENAI_API_KEY)
claude = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
genai.configure(api_key=GOOGLE_API_KEY)

# LLM models
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-latest"
GEMINI_MODEL = "gemini-2.0-flash-exp"  # "gemini-1.5-pro"

----
## LLM Call Functions
Below are some generic LLM functions that will be utilized by the two Gradio apps that we will be seeing shortly.

In [ ]:
# --------- Providers ---------
def call_anthropic(prompt: str, temperature: float = 0.2, max_tokens: int = 4096) -> str:
    print(datetime.datetime.now(), '-->', 'call_anthropic()')
    resp = client.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=max_tokens,
        temperature=temperature,
        system=SYSTEM_PROMPT,
        messages=[{"role": "user", "content": prompt}],
    )
    parts = []
    for block in resp.content:
        if getattr(block, "type", None) == "text":
            parts.append(block.text)
    return "".join(parts).strip() or "# Empty response from Anthropic."

def call_openai(prompt: str, temperature: float = 0.2, max_tokens: int = 4096) -> str:
    print(datetime.datetime.now(), '-->', 'call_openai()')
    resp = openai.chat.completions.create(
        model=OPENAI_MODEL,
        temperature=temperature,
        max_tokens=max_tokens,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
    )
    text = resp.choices[0].message.content or ""
    return text.strip().removeprefix("```python").removesuffix("```").strip() or "# Empty response from OpenAI."

def call_gemini(prompt: str, temperature: float = 0.2, max_tokens: int = 4096) -> str:
    print(datetime.datetime.now(), '-->', 'call_gemini()')
    model = genai.GenerativeModel(GEMINI_MODEL, system_instruction=SYSTEM_PROMPT)
    resp = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            temperature=temperature,
            max_output_tokens=max_tokens,
        ),
    )
    text = (resp.text or "").strip()
    return text.removeprefix("```python").removesuffix("```").strip() or "# Empty response from Gemini."

# --------- Router ---------
def router_func(prompt: str, provider: str, temperature: float = 0.2) -> str:
    try:
        if provider == "Anthropic Claude":
            return call_anthropic(prompt, temperature)
        elif provider == "OpenAI GPT":
            return call_openai(prompt, temperature)
        elif provider == "Google Gemini":
            return call_gemini(prompt, temperature)
        else:
            return "# Unknown provider selected."
    except Exception as e:
        return f"# Error: {e}\n# Check the selected provider's SDK, model name, and API key."

----
## Gradio App 1: Add comments and docstrings to Python code
Here we will build a Gradio app that takes a Python script and adds comments/docstrings to it.

In [ ]:
SYSTEM_PROMPT = """\
You are a Python code reviewer. Add clear, PEP 257-compliant docstrings to all public
functions/classes/modules and concise inline comments where helpful. Preserve logic and
formatting as much as possible. If code has issues, fix only what is necessary to make
the docstrings and comments accurate. Return *only* the fully updated Python code.
"""

# A compact instruction that encourages structured, deterministic output.
USER_INSTRUCTION_TEMPLATE = """\
Rewrite the following Python script by adding:
1) A module-level docstring at the top summarizing purpose and usage.
2) Docstrings for public functions/classes with arguments, returns, raises.
3) Helpful inline comments for non-obvious logic.
4) Keep imports, names, and structure stable unless a tiny refactor improves clarity.
5) Do not wrap code in triple backticks; return only the code.

Python script:
{code}
"""

# --------- Router ---------
def annotate_code(py_code: str, provider: str, temperature: float = 0.2) -> str:
    if not py_code or not py_code.strip():
        return "# Please paste a Python script on the left."
    prompt = USER_INSTRUCTION_TEMPLATE.format(code=py_code)
    return router_func(prompt, provider, temperature)

# --------- Gradio UI ---------
with gr.Blocks(title="Docstring & Comments Enhancer") as demo:
    gr.Markdown("## Python Comment & Docstring Enhancer")
    with gr.Row():
        with gr.Column():
            provider = gr.Dropdown(
                choices=[
                    "Anthropic Claude",
                    "OpenAI GPT",
                    "Google Gemini",
                ],
                value="Anthropic Claude",
                label="LLM Provider / Model",
            )
            code_in = gr.Code(
                label="Paste your Python script",
                language="python",
                lines=24,
                value=textwrap.dedent(
                    """\
                    import math

                    def hypotenuse(a, b):
                        return (a**2 + b**2) ** 0.5

                    class Accumulator:
                        def __init__(self):
                            self.total = 0
                        def add(self, x):
                            self.total += x
                            return self.total

                    if __name__ == "__main__":
                        print(hypotenuse(3,4))
                        acc = Accumulator()
                        for i in range(5):
                            print(acc.add(i))
                    """
                ),
            )
            temperature = gr.Slider(0.0, 1.0, value=0.2, step=0.05, label="Temperature")
            run_btn = gr.Button("Enhance", variant="primary")
        with gr.Column():
            code_out = gr.Code(label="Commented & Documented Code", language="python", lines=24)

    run_btn.click(
        fn=lambda c, p, t: annotate_code(c, p, t),
        inputs=[code_in, provider, temperature],
        outputs=code_out,
    )

if __name__ == "__main__":
    demo.launch()

----
## Gradio App 2: Write unit test cases for given functions
Here we will build a Gradio app that write unit test cases for given python functions.

In [ ]:
# Coming soon...